In [9]:
# First we grab the model and the unembedding weight matrix
import torch
from easy_transformer import EasyTransformer

#device = 'cuda' if torch.cuda.is_available() else 'cpu'
device = 'cpu'
print(f"Using {device} device")
torch.set_grad_enabled(False)

model = EasyTransformer.from_pretrained('gpt2').to(device)

# Convenience function for decoding token
decode = model.tokenizer.decode
encode = model.tokenizer.encode

M_to_V = model.unembed.W_U.data
d_M = model.cfg.d_model
d_V = model.cfg.d_vocab

Using cpu device
Loading model: gpt2


Using pad_token, but it is not set yet.


Moving model to device:  cpu
Finished loading pretrained model gpt2 into EasyTransformer!
Moving model to device:  cpu


In [35]:
from sklearn.decomposition import PCA

V_to_M = torch.nn.functional.normalize(torch.linalg.pinv(M_to_V), dim=1)

pca_model = PCA(n_components=d_M)
M_to_D = torch.tensor(pca_model.fit_transform(M_to_V), dtype=torch.float)
D_to_V = torch.tensor(pca_model.components_, dtype=torch.float)

norms = torch.linalg.vector_norm(M_to_D, dim=0)
#M_to_D = M_to_D / norms.reshape(1, d_D).expand(d_M, d_D)
#D_to_V = D_to_V * norms.reshape(d_D, 1).expand(d_D, d_V)

#print(M_to_V)
#print(torch.matmul(M_to_D, D_to_V))


V_to_D = torch.matmul(V_to_M, M_to_D)
d_D = d_M

print(V_to_M.shape, M_to_D.shape, D_to_V.shape, M_to_V.shape, V_to_D.shape)

torch.Size([50257, 768]) torch.Size([768, 768]) torch.Size([768, 50257]) torch.Size([768, 50257]) torch.Size([50257, 768])


In [36]:
example_tokens = [' cat', ' war', ' banana', ' bat', ' lie']

def print_similar(heading, td):
    print(heading, torch.linalg.vector_norm(td))
    similarity_vec = torch.matmul(V_to_D, td)
    values = [(v.item(), i) for i,v in enumerate(similarity_vec)]
    values.sort(reverse=True)
    for v,i in values[:20]:
        print(f'  {decode(i):20}', f'{v:20}')
    print()

for tok in example_tokens:
    t, = encode(tok)
    td = V_to_D[t,:]
    print_similar(tok, td)

#generator = torch.random.manual_seed(12345)
#for i in range(3):
#    tr = torch.zeros((d_D))
#    tr.normal_(generator=generator)
#    tr = tr ** 5
#    print_similar('(random)', tr)

 cat tensor(25.1970)
   cat                       634.8876953125
   cats                  404.85211181640625
   Cat                        401.826171875
  Cat                    327.85736083984375
  cat                     311.1290283203125
   Cats                   273.6062316894531
   dog                   254.74285888671875
   CAT                      220.56103515625
  cats                   218.95774841308594
   catcher                 211.718994140625
   kittens               210.29351806640625
   kitten                202.44485473632812
   dogs                  166.30856323242188
   animal                 162.6132049560547
   pet                   160.79998779296875
   goat                    159.751220703125
   Dog                   155.82252502441406
   rabbit                155.04403686523438
   tiger                  149.2018280029297
   fel                   142.49505615234375

 war tensor(23.9921)
   war                      575.61865234375
   War                   425.1563